Description and demonstration of common Keras layers. For more information see [Keras layers](https://keras.io/layers/about-keras-layers/).

## Dense layer

A dense layer calculates $ output = activation(dot(W, input) + b)$

Popular activation functions are:
 * linear
 * sigmoid
 * softmax
 * relu

In [9]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model


weights = np.array([[1,2,3,4,5],[1,2,3,4,5]])
bias = np.array([0,0,0,0,0])
weights_and_bias = (weights, bias)

inputs = Input(shape=(2,)) # each X_i has 2 dimensions
outputs = Dense(5, activation='linear', weights=weights_and_bias)(inputs) 

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='sgd', loss='mse')

print('Input shape', model.input.shape)
print('Output shape', model.output.shape)

model_weights = model.layers[1].get_weights()
print('\nTrainable parameters:')
print("W.shape:", model_weights[0].shape)
print("b.shape:", model_weights[1].shape)


Input shape (?, 2)
Output shape (?, 5)

Trainable parameters:
W.shape: (2, 5)
b.shape: (5,)


## TODO Dropout layer

## Embedding layer

In [ ]:
import numpy as np
from keras.layers import Input, Dense
from keras.layers.embeddings import Embedding
from keras.models import Model

max_sequence = 5
embedding_dims = 10

# Size of the vocabulary. The assumption is that indexing starts with 0
# and is consequtive.
vocab_size = 3

inputs = Input(shape=(max_sequence,), dtype='int32') # each X_i is a sequence of 'max_sequence' integers
outputs = Embedding(vocab_size, embedding_dims, input_length=max_sequence)(inputs)

model = Model([inputs], [outputs])
model.compile(optimizer='sgd', loss='mse')

model.predict( np.array([[0,1,1,1,2]]) )

# An embedding layer is like a lookup table. The values in the input 
# vector are used as indices in the internal weights matrix.

## Lambda layer
Simple way to add functionality to a model. Best used for stateless functions. For stateful functions it is better to implement a separate layer.

In [5]:
import numpy as np
from keras.layers import Input, Concatenate, Lambda, Embedding, Average, Dense, Add
from keras.models import Model


inputs = Input(shape=(4,4), dtype='float32')
first_row = Lambda(lambda x: x[:,0,:])(inputs)
first_column = Lambda(lambda x: x[:,:,0])(inputs)
doubled = Lambda(lambda x: x*2.0)(inputs)

model = Model(inputs=inputs, outputs=[first_row, first_column, doubled])
model.compile(optimizer='sgd', loss='mse')

x = np.array([
    [
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3],
    [4,4,4,4],        
    ]
], dtype=np.float32)

x,y,z = model.predict(x)
print(x)
print(y)
print(z)

[[1. 1. 1. 1.]]
[[1. 2. 3. 4.]]
[[[2. 2. 2. 2.]
  [4. 4. 4. 4.]
  [6. 6. 6. 6.]
  [8. 8. 8. 8.]]]


## Concatenate and Average layer

In [2]:
import numpy as np
from keras.layers import Input, Concatenate, Lambda, Embedding, Average, Dense, Add
from keras.models import Model


inputs = Input(shape=(4,4), dtype='float32')
word_vector_rows = [Lambda(lambda x: x[:,i,:])(inputs) for i in range(win_size)]
concat_out = Concatenate()(word_vector_rows)
avg_out = Average()(word_vector_rows)
sum_out = Add()(word_vector_rows)


model = Model(inputs=inputs, outputs=[concat_out, avg_out, sum_out])

model.compile(optimizer='sgd', loss='mse')

x = np.array([
    [
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3],
    [4,4,4,4],        
    ]
], dtype=np.float32)

print(x.shape)

concat_out_val, avg_out_val, sum_out_val = model.predict(x)
print('concat_out', concat_out_val)
print('avg_out', avg_out_val)
print('sum_out', sum_out_val)

(1, 4, 4)
concat_out [[1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4.]]
avg_out [[2.5 2.5 2.5 2.5]]
sum_out [[10. 10. 10. 10.]]


## Softmax layer

For a vector $x$ with elements $x_i$ the softmax function calculates:

$ softmax(x_i) = \frac{e^{x_i}}{\sum_i {e^{x_i}}} $


In [14]:
import numpy as np
from keras.layers import Input, Concatenate, Lambda, Embedding, Average, Dense, Add, Activation
from keras.models import Model

nb_classes = 4
inputs = Input(shape=(nb_classes,), dtype='float32')
softmax = Activation('softmax')(inputs)
model = Model(inputs=inputs, outputs=softmax)
model.compile(optimizer='sgd', loss='mse')

x = np.array([[8.0, 2.0, 9.0, 3.0]])
probs = model.predict(x)
print('probabilities:', probs)
print('predicted class:', np.argmax(probs))

probabilities: [[2.6827645e-01 6.6499080e-04 7.2925097e-01 1.8076325e-03]]
predicted class: 2


## Cross Entropy loss

Input to the cross entropy function **must** be a probability distribution.

$ cross\_entropy(y\_true, y\_pred) = -log(y\_pred_{y\_true}) $

There are multiple implementations of cross entropy:
 * categorical vs. binary
 * sparse vs. one-hot encoded
 
In Tensorflow you usually use optimized functions that combine softmax and cross entropy.

In [26]:
import keras
from keras import backend as K
from keras.losses import sparse_categorical_crossentropy

nb_classes = 5
y_true = K.variable(value=np.array([1]), dtype='float32')
y_pred = K.variable(value=np.array([[0.01, 0.01, 0.96, 0.01, 0.01]]), dtype='float32')
loss_fn = keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

loss = K.eval(loss_fn)
print('cross entropy loss:', loss[0])

cross entropy loss: 4.6051702
